In [1]:
from pyspark import SparkContext
from pyspark.sql import *
import os
import glob
from time import time
from delta import *

In [2]:
# sc = SparkContext.getOrCreate()
# spark = SparkSession(sc)
builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
start_time = time()

**Read Data from Landing Zone**

In [4]:
all_generations_df_json = spark.read.option("multiLine", "true").json("landing-zone/2022-10-03 09-43-29/json/items/generations/*.json")
all_pokemons_df_json = spark.read.option("multiLine", "true").json("landing-zone/2022-10-03 09-43-29/json/items/pokemons/*.json")
all_stats_df_json = spark.read.option("multiLine", "true").json("landing-zone/2022-10-03 09-43-29/json/items/stats/*.json")
all_types_df_json = spark.read.option("multiLine", "true").json("landing-zone/2022-10-03 09-43-29/json/items/types/*.json")
all_version_groups_df_json = spark.read.option("multiLine", "true").json("landing-zone/2022-10-03 09-43-29/json/items/version_groups/*.json")
all_versions_df_json = spark.read.option("multiLine", "true").json("landing-zone/2022-10-03 09-43-29/json/items/versions/*.json")

In [5]:
checkpoint_1_time = time()
print("Time taken to read All Landing Data JSON: " + str(checkpoint_1_time - start_time))

Time taken to read All Landing Data JSON: 249.21813344955444


**Write Data in Silver Zone**

In [6]:
all_generations_df_json.write.format("delta").mode("overwrite").save(f"silver-zone/items/generations.delta")
all_pokemons_df_json.write.format("delta").mode("overwrite").save(f"silver-zone/items/pokemons.delta")
all_stats_df_json.write.format("delta").mode("overwrite").save(f"silver-zone/items/stats.delta")
all_types_df_json.write.format("delta").mode("overwrite").save(f"silver-zone/items/types.delta")
all_version_groups_df_json.write.format("delta").mode("overwrite").save(f"silver-zone/items/version_groups.delta")
all_versions_df_json.write.format("delta").mode("overwrite").save(f"silver-zone/items/versions.delta")

In [7]:
checkpoint_2_time = time()
print("Time taken to Write Delta in Silver Zone: " + str(checkpoint_2_time - checkpoint_1_time))

Time taken to Write Delta in Silver Zone: 60.88992643356323


In [8]:
all_generations_df_json.coalesce(1).write.format("parquet").mode("overwrite").save(f"silver-zone/items/generations.parquet")
all_pokemons_df_json.coalesce(1).write.format("parquet").mode("overwrite").save(f"silver-zone/items/pokemons.parquet")
all_stats_df_json.coalesce(1).write.format("parquet").mode("overwrite").save(f"silver-zone/items/stats.parquet")
all_types_df_json.coalesce(1).write.format("parquet").mode("overwrite").save(f"silver-zone/items/types.parquet")
all_version_groups_df_json.coalesce(1).write.format("parquet").mode("overwrite").save(f"silver-zone/items/version_groups.parquet")
all_versions_df_json.coalesce(1).write.format("parquet").mode("overwrite").save(f"silver-zone/items/versions.parquet")

In [9]:
checkpoint_3_time = time()
print("Time taken to Write Parquet in Silver Zone: " + str(checkpoint_3_time - checkpoint_2_time))

Time taken to Write Parquet in Silver Zone: 92.78850507736206


In [ ]:
# deltaTable = DeltaTable.forPath(spark, f"bronze-zone/items/generations.delta")
# deltaTable.vacuum()
# deltaTable.history().show()